# Fund Strategy - Signal Event 0003

## 1. Import Library

In [7]:
import dai
import random
import pandas as pd

## 2. Prepare Strategy Data

In [8]:
sd = '2015-12-31'
ed = '2026-02-13'

In [9]:
instrument_list = ['513030.SH']

In [10]:
sql = f"""
SELECT
    date,
    instrument,
    m_ta_kdj_k(high, low, close) AS K,
    m_ta_kdj_d(high, low, close) AS D,
    m_ta_kdj_j(high, low, close) AS J,
    IF(K > D AND m_lag(K, 1) < m_lag(D, 1), 1, 0) AS signal_buy,
    IF(K < D AND m_lag(K, 1) > m_lag(D, 1), 1, 0) AS signal_sell,
FROM cn_fund_bar1d
WHERE instrument IN {str(tuple(instrument_list))}
QUALIFY (signal_buy = 1 OR signal_sell = 1)
"""

In [11]:
df = dai.query(sql, filters = {"date":[sd, ed]}).df()
df

,date,instrument,K,D,J,signal_buy,signal_sell
0,2024-06-12,513030.SH,42.714283,49.453275,29.236298,0,1
1,2024-06-21,513030.SH,30.880427,30.520503,31.600274,1,0
2,2024-06-24,513030.SH,29.071800,30.037602,27.140195,0,1
3,2024-06-25,513030.SH,35.138775,31.737993,41.940340,1,0
4,2024-07-10,513030.SH,73.249512,77.549787,64.648963,0,1
...,...,...,...,...,...,...,...
381,2024-04-22,513030.SH,24.924781,20.386640,34.001062,1,0
382,2024-05-17,513030.SH,86.634648,90.017839,79.868265,0,1
383,2024-05-28,513030.SH,59.348153,54.458233,69.127993,1,0
384,2024-05-29,513030.SH,52.385948,53.767471,49.622902,0,1


## 3. Backtest Strategy

In [12]:
from bigmodule import M

def BigTrader_Initialize(context):
    from bigtrader.finance.commission import PerOrder
    context.set_commission(PerOrder(buy_cost=0.0003, sell_cost=0.0013, min_cost=5))
    context.holding_days = 3
    context.is_trade_by_weight = True
    context.weight = 1 / len(context.data['instrument'].unique())

def BigTrader_Before_Trading(context, data):
    pass

def BigTrader_Handle_Tick(context, tick):
    pass

def BigTrader_Handle_Data(context, data):

    df_now = context.data[context.data["date"] == data.current_dt.strftime("%Y-%m-%d")]

    if len(df_now) == 0:
        return

    instruments_buy  = set(df_now[df_now["signal_buy"]  == 1]["instrument"])
    instruments_sell = set(df_now[df_now["signal_sell"] == 1]["instrument"])
    instruments_hold = set(context.get_account_positions().keys())

    for instrument in instruments_buy - instruments_hold:
        if context.is_trade_by_weight:
            context.order_target_percent(instrument, context.weight)
        else:
            context.order(instrument, 100)
    
    for instrument in instruments_sell:
        context.order_target_percent(instrument, 0)

    for instrument in instruments_hold:
        if (data.current_dt - context.get_position(instrument).last_sale_date).days >= context.holding_days:
            context.order_target_percent(instrument, 0)

def BigTrader_Handle_Trade(context, trade):
    pass

def BigTrader_Handle_Order(context, order):
    pass

def BigTrader_After_Trading(context, data):
    pass

BigTrader = M.bigtrader.v34(
    
    data = df,
    
    start_date = """""",
    end_date   = """""",
    
    initialize           = BigTrader_Initialize,
    before_trading_start = BigTrader_Before_Trading,
    handle_tick          = BigTrader_Handle_Tick,
    handle_data          = BigTrader_Handle_Data,
    handle_trade         = BigTrader_Handle_Trade,
    handle_order         = BigTrader_Handle_Order,
    after_trading        = BigTrader_After_Trading,
    
    capital_base = 1000000 + random.uniform(0, 10),
    frequency="""daily""",
    product_type="""自动""",
    rebalance_period_type="""交易日""",
    rebalance_period_days="""1""",
    rebalance_period_roll_forward=True,
    backtest_engine_mode="""标准模式""",
    before_start_days=0,
    volume_limit=1,
    order_price_field_buy="""open""",
    order_price_field_sell="""open""",
    benchmark="""沪深300指数""",
    
    plot_charts=True,
    debug=False,
    backtest_only=False,
    m_name="""BigTrader"""
) 

[2026-02-19 15:15:28] [info     ] bigtrader.v34 开始运行 ..
[2026-02-19 15:15:28] [info     ] 2016-01-26, 2025-01-15, , , instruments=1
[2026-02-19 15:15:28] [info     ] bigtrader module V2.2.0
[2026-02-19 15:15:28] [info     ] bigtrader engine v0.1.0.post9+g7a244b6 2026-02-10
[2026-02-19 15:15:31] [info     ] backtest done, raw_perf_ds:dai.DataSource("_93de071550f54083be1bd45b69e992b4")


[2026-02-19 15:15:31] [info     ] bigtrader.v34 运行完成 [3.053s].
